In [23]:
# imports
from google.colab import drive
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time

In [24]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive"
infile = open(DATA_PATH+'/combined_cleaned.pkl','rb')
#infile = open(DATA_PATH+'/combined_df.pkl','rb')
df = pickle.load(infile)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
df

,Email Text,Email Type
0,": 6 . 1100 , disc : uniformitarianism , : 1086...",Safe Email
1,side galicismos galicismo spanish term name im...,Safe Email
2,: equistar deal ticket still available assist ...,Safe Email
3,"hello hot lil horny toy . one dream , open min...",Malicious Email
4,software incredibly low price 86 lower . drape...,Malicious Email
...,...,...
23816,subject : put 10 ft transport volume decreased...,Safe Email
23817,subject : 3 4 2000 following noms hpl ' take e...,Safe Email
23818,"subject : calpine daily gas nomination julie ,...",Safe Email
23819,subject : industrial worksheet august 2000 act...,Safe Email


In [27]:
# unique_types = df['Email Text'].apply(type).unique()
# print(unique_types)

# # Find rows where 'email_text' is not a string
# non_string_entries = df[df['Email Text'].apply(lambda x: not isinstance(x, str))]

# # Display these rows to understand what they contain
# print(non_string_entries)
# # remove NaNs
# df = df.dropna(subset=['Email Text'])

In [28]:
def preprocess_text(text):
    # Cleaning: remove special characters and extra spaces
    # text = re.sub(r'[\t\n\r]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,;\'\":-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Stop Words Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]

    # Rejoin tokens into a single string
    return ' '.join(tokens)

# because this one is already cleaned
# df.loc[:, 'email_text_cleaned'] = df['Email Text'].apply(preprocess_text)

In [29]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC

In [31]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Email Text'], df['Email Type'], test_size=0.3, random_state=42)

In [32]:
from scipy.stats import uniform

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', LinearSVC(dual=False, random_state=42))
])

# Define the parameter distributions to sample from
param_distributions = {
    'classifier__C': uniform(0.01, 10),  # Uniform distribution from 0.01 to 10
    'classifier__max_iter': [100, 200, 300, 400, 500],
    'classifier__tol': uniform(1e-5, 1e-3)  # Tolerance for stopping criteria
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC

# Initialize the LinearSVC model
linear_svc = LinearSVC(dual=False, random_state=42)  # dual=False when n_samples > n_features

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_distributions, n_iter=50, cv=3, verbose=4, random_state=42, n_jobs=-1)

# Assuming X and y are your features and labels
# Fit the RandomizedSearchCV object to find the best parameters
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# To use the best model found:
best_model = random_search.best_estimator_

# You can further evaluate this model on a separate test set or use it for predictions

In [ ]:
# # Define a list of pipelines for different models
# pipeline = Pipeline([
#         ('tfidf', TfidfVectorizer(stop_words='english')),
#         ('classifier', LinearSVC())
#     ])

In [ ]:
# # Train the model on the training data
# pipeline.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print(report)

In [ ]:
from joblib import dump, load

# Save the model to a file
model_filename = DATA_PATH + '/email_classifier_linearSVC.joblib'
dump(best_model, model_filename)

print(f"Model saved to {model_filename}")

In [ ]:
# Example list of test emails and their correct classifications
test_emails = [
    #1
    {"text": "Personal Assistant Needed\nYou are invited to participate in a Part-time work offer for current\nstaff/students. For more information About the task CLICK HERE\nApplication will be received and you will get a response between\n24- 48 hours.", "label": "Malicious Email"},
    #2
    {"text": """Hello Feng Du,
Thank you for shopping with us. We wanted to let you know that LaGOnlinestore has shipped your item(s) on 11/15 and that this completes your order. If you need to return an item or manage other orders, please visit Your Orders on Amazon.co.jp. You can print out a receipt here.""", "label": "Safe Email"},
    #3
    {"text": """Verify and sign in
Hi there,

Verify yourself below to sign in to your Booking.com account for fengdu88@gmail.com.

The link can only be used once and expires in 10 minutes if you don’t use it.""", "label": "Safe Email"},
    #4
     {"text": """Hi Feng, you have a chat pending request from 马雯煊


Introduction from 马雯煊:
I'm very interested in you, can you talk with me

To confirm or delete this request, or block this user, follow the link below:

https://www.conversationexchange.com/members/friendRequestPending.php?lg=en

The Conversation Exchange Team""", "label": "Safe Email"},
    #5
     {"text": """Volley6'ers,

Only 72hrs remaining in our Captain's Pre-sale!!

Our pre-sale is now available to current captains and Public Registration (incl. individual player registration) will open up on Monday, March 4th at 10am. Your team space can be reserved with a deposit of $150.

 Please keep in mind that the deposit must be paid during the captain's pre-sale to secure your spot.""", "label": "Safe Email"},
    #6
     {"text": """Hello,
This is Adam from Microsoft Customer Centre.
We have found out your account has been hacked.
Please call to our support centre right away: 831-385-4871.""", "label": "Malicious Email"},
    #7
     {"text": """Subject: Action Required: Verify Your Microsoft Account Now

Dear User,

We've detected suspicious activity on your Microsoft account and require immediate verification to prevent unauthorized access. Please click the link below to verify your identity, or your account will face temporary suspension.

[Verify My Account]

Thank you,
Microsoft Security Team.""", "label": "Malicious Email"},
    #8
     {"text": """Subject: Amazon Delivery Failure - Immediate Response Required

Hello,

We encountered an issue delivering your recent Amazon order. To avoid return processing, please click here to confirm your shipping details.

[Confirm Shipping Details]

Best Wishes,
Amazon Delivery Services""", "label": "Malicious Email"},
    #9
     {"text": """Subject: Netflix Subscription Renewal Failed

We were unable to renew your Netflix subscription as your payment was declined. Please update your payment details to continue enjoying your subscription without interruption.

[Update Payment Details]

Best,
Netflix Support Team""", "label": "Malicious Email"},
    {"text": """Subject: You've Received a Shared Document on Google Drive

A document has been shared with you on Google Drive. Click below to view the document. Please enter your Google credentials to access the file.

[View Document]

Best,
Google Drive Team""", "label": "Malicious Email"},
    #10
     {"text": """Subject: PayPal Payment Not Processed Successfully

Dear User,

Your recent PayPal payment could not be processed. To avoid service interruption, please update your payment information promptly.

[Update Payment Information]

Thank you,
PayPal Billing Department""", "label": "Malicious Email"},
    #11
     {"text": """Subject: Apple Security Alert: Your iCloud Account Has Been Breached

Immediate action is required to secure your iCloud account. We've detected unauthorized access and need you to verify your credentials.

[Secure My Account]

Sincerely,
Apple Support Team""", "label": "Malicious Email"},
    #12
     {"text": """Booking.com
Confirm Your Email Address

Hi there,

We're excited to have you with us! To get started with your Booking.com account for [Recipient's Email Address], please verify your email address below. This ensures that we can communicate important booking details and updates.

The link below is for your use only and will expire in 10 minutes if not used.

[Confirm My Email]

How does email confirmation work?
This secure, one-time URL verifies that you have access to this email address. Verifying your email doesn't affect your password or login methods but adds an extra layer of security to your account.

If you did not sign up for a Booking.com account, no further action is required on your part. You can safely disregard this email.

Having trouble with the button above? You can also confirm your email by visiting this link: [Verification Link]

Safe travels,
The Booking.com Team""", "label": "Safe Email"},
    #13
     {"text": """Dear Mr./Ms. Feng Du,

We hope this message finds you well. Thank you for continuing to choose Sakura Mobile for your communication needs. Enclosed in this email is the invoice for your mobile service usage for the month of January 2024.

Invoice Summary:

Booking ID: QMDA-974625
Invoice Number: QMDA-974625-202401
Issue Date: February 25, 2024
Payment Due Date: February 29, 2024
Billing Address: Otakibashi Pacifica Bld. 9th Fl. 7-10-18, Nishishinjuku, Shinjuku-ku, Tokyo, Japan
Contact Email: contact@sakuramobile.jp
Website: www.sakuramobile.jp
Phone: 03-5937-5869
Fax: 050-3737-7625
Account Details:

Customer: Feng Du
Email: fengdu88@gmail.com
Charges for January 2024:

Monthly Fee (Voice & Data SIM 25GB): ¥3,980
International Service: ¥0
Domestic Call Charges: ¥0
SakuraDial (Domestic): ¥0
SakuraDial (International): ¥0
Domestic SMS Charges: ¥0
Subtotal: ¥3,980
V.A.T (10%): ¥398
Total Due: ¥4,378
Payment Information:
Your payment for the above total will be automatically processed using the card registered with us at the end of February. To ensure a smooth transaction, please verify that your registered card has sufficient funds available. In the event of a transaction failure, please be aware that your mobile line may be temporarily suspended until the payment issue is resolved.

For any inquiries or assistance, please do not hesitate to contact us at the email or phone number provided above. Our team is here to support you with any needs related to your Sakura Mobile service.

Thank you once again for choosing Sakura Mobile. We appreciate your business and look forward to continuing to serve you.

Warm regards,

The Sakura Mobile Team""", "label": "Safe Email"},
    #14
     {"text": """SHighbury Park Dental
Your Upcoming Dental Appointment

Dear Feng,

We hope this message finds you well. Just a gentle reminder that your upcoming dental appointment at Highbury Park Dental is scheduled for:

Date: Monday, March 4, 2024
Time: 5:00 PM

Location:
Highbury Park Dental
1055 Greenbank Rd., Unit 5
Nepean, ON K2J 1V8
Phone: (613) 440-7040

Click Here to Confirm Your Appointment

To make it easier for you, we've included options to add this appointment to your digital calendar:

Add to Google Calendar
Add to iCal
Add to Outlook
Cancellation Policy Reminder:
We understand that plans can change, which is why we kindly request that you inform us at least 48 hours in advance if you need to reschedule your appointment. Please note, missing an appointment without prior notification will result in a $50.00 missed appointment fee, which cannot be billed to insurance and will be your direct responsibility.

We look forward to seeing you soon and providing you with the highest quality dental care. If you have any questions or need further assistance, please don't hesitate to contact our office at (613) 440-7040.

Thank you for choosing Highbury Park Dental for your dental health needs.

Warm regards,

The Team at Highbury Park Dental
1055 Greenbank Rd., Unit 5
Nepean, ON K2J 1V8
(613) 440-70""", "label": "Safe Email"},
    #15
     {"text": """Dear Nintendo Switch Online Member,

This email serves as a reminder that your Nintendo Switch Online individual 12-month (365-day) membership is set to renew automatically soon. We're excited to continue providing you with great online features and exclusive content.

Renewal Details:

Membership Term: 12 months (365 days)
Renewal Price: $24.99 (subject to applicable taxes)
Automatic Renewal Deadline:
You have until 03/19/2024, 16:21 PDT to manage your subscription settings. If you wish to turn off automatic renewal, please do so before the deadline.

Managing Your Membership:
To view your membership details or modify your subscription settings, please visit your Nintendo Account settings or follow the link below:

Manage My Membership

We appreciate your continued support and look forward to enhancing your gaming experience on the Nintendo Switch.

If you have any questions or require further assistance, please do not hesitate to contact our support team.

Happy Gaming,
Nintendo Customer Support""", "label": "Safe Email"},
    #16
     {"text": """asdgkhasduigdfgjoidjgoisdfjgoisdfjogjdsiojdf""", "label": "Malicious Email"}
    # Add more test emails and their correct labels here
]


In [ ]:
# Assuming 'selected_pipeline' is your chosen pipeline
for i, test_email in enumerate(test_emails, start=1):
    test_text = test_email["text"]
    correct_label = test_email["label"]
    # print(test_text)
    # Make a prediction

    predicted_label = best_model.predict([preprocess_text(test_text)])
    #predicted_label = pipeline.predict([test_text])

    # Check if the prediction is correct
    if predicted_label == correct_label:
        print(f"Test Case {i}: Correctly classified as '{predicted_label}'")
    else:
        print(f"Test Case {i}: Incorrectly classified as '{predicted_label}' (Correct: '{correct_label}')")

correct_predictions = 0
total_tests = len(test_emails)  # Total number of test cases

for i, test_email in enumerate(test_emails, start=1):
    test_text = test_email["text"]
    correct_label = test_email["label"]

    # Make a prediction and wrap the single string in a list
    predicted_label = best_model.predict([preprocess_text(test_text)])
    #predicted_label = pipeline.predict([test_text])

    # Check if the prediction is correct
    if predicted_label == correct_label:
        correct_predictions += 1  # Increment the correct predictions counter

# Now `correct_predictions` holds the number of correct predictions
accuracy = correct_predictions / total_tests
print(f"Accuracy: {accuracy:.4f}")

